# Метод Флетчера-Ривза и метод Полака-Рибьера

In [1]:
import numpy as np
import numpy.linalg as lg

## функция и ее градиенты

In [2]:
def fx(x): 
    return np.exp(sum(np.square(x)))
def gradfx(x):
    return fx(x) * (2*x) 
def grad2fx(x):
    return fx(x) * 2 * (np.eye(len(x)) + 2 * x.reshape(len(x),1) * x)

# овражная функция
def hx(x): 
    x1 = x
    x1[0] = x[0]/10
    return np.exp(sum(np.square(x1)))
def gradhx(x_):
    x = x_
    x[0] = x[0]/10
    return fx(x) * (2*x) 
def grad2hx(x_):
    x = x_
    x[0] = x[0]/10
    return fx(x) * 2 * (np.eye(len(x)) + 2 * x.reshape(len(x),1) * x)

#альтернативная ДВУМЕРНАЯ функция ( для проверки)
def gx(x):
    k = np.array([3, 1])
    return sum(np.square(k * x))
def gradgx(x):
    k = np.array([18, 2])
    return x * k
def grad2gx(x):
    return np.array([[18 ,0],[0, 2]])

## Вспомогательные функции

### "Умные" производные

In [19]:
def func(f, x0, gradx0):
    def fun(a):
        A = x0 - gradx0 * a
        return f(A)
    return fun

def der1_e(f_, x0, a, gradx0, e):
    f_(x0)
    f = func(f_, x0, gradx0)
    def der(a):
        return (f(a + e) - f(a - e)) / (2*e)
    return der

def der2_e(f_, x0, a, gradx0, e):
    df = der1_e(f_, x0, a, gradx0, e)
    def der2(a):
        return (df(a + e) - df(a - e))/ (2 *e)
    return der2

In [20]:
def grad_descent_a1(f, x0, vec, e): 
    a = 0
    df = der1_e(f, x0, a, vec, e)
    while e < abs(df(a)):
        d2f = der2_e(f, x0, a, vec, e)
        a = a - df(a)/d2f(a)
        df = der1_e(f, x0, a, vec, e)
    return a

## Флетчер-Ривз

In [1]:
def Flatcher_Rivz(f, grad, x, e):
    n = 0
    d = -grad(x)
    while abs(lg.norm(grad(x))) > e:
        print('-----', lg.norm(grad(x)))
        n+=1
        print('')
        print('------[----', n, '----]------')
        
        a = grad_descent_a1(f, x, -d, e/100)
        if(n % len(x) != 0):
            x += a * d 
            d = -grad(x)
        else:
            x += a*d
            
            b = (lg.norm(grad(x - a*d))/lg.norm(grad(x)))**2
            d = -grad(x) + b*d
        
        print('  x  = ', x)
        print('f(x) = ', f(x))
        
    print('сделано ', n, ' шагов')
    
    return x

In [22]:
x0 = np.array([1.0, 1.2, 1.3])
e = 0.0005
x_min = Flatcher_Rivz(fx, gradfx, x0, e)
print(round(fx(x_min), 3), x_min)

----- 252.72094202884531

------[---- 1 ----]------
  x  =  [3.65776631e-10 4.38932002e-10 4.75509632e-10]
f(x) =  1.0
сделано  1  шагов
1.0 [3.65776631e-10 4.38932002e-10 4.75509632e-10]


In [36]:
x0 = np.array([5.,1.])
e = 0.05
x_min = Flatcher_Rivz(gx, gradgx, x0, e)
print(round(gx(x_min), 3), x_min)


------[---- 1 ----]------
  x  =  [0.02030533 0.88934012]
f(x) =  0.7946366053438533

------[---- 2 ----]------
  x  =  [-0.11761858  0.21813566]
f(x) =  0.17209034285214284

------[---- 3 ----]------
  x  =  [0.00388943 0.19309688]
f(x) =  0.03742255397659784
сделано  3  шагов
0.037 [0.00388943 0.19309688]


In [40]:
x0 = np.array([5.,1.,1.])
e = 0.05
x_min = Flatcher_Rivz(hx, gradhx, x0, e)
print(round(hx(x_min), 3), x_min)


------[---- 1 ----]------
  x  =  [-4.97003767e-07  5.99246549e-03  5.99246549e-03]
f(x) =  1.0000718218643334
сделано  1  шагов
1.0 [-4.97003767e-10  5.99246549e-03  5.99246549e-03]


## Полак-Рибьер

In [31]:
def Polak_Ribier(f, grad, x, e):
    n = 0
    d = -grad(x)
    while abs(lg.norm(grad(x))) > e * 10:
        n+=1
        print('')
        print('------[----', n, '----]------')
        
        a = grad_descent_a1(f, x, -d, e/100)
        if(n % len(x) == 0):
            x += a * d 
            d = -grad(x)
        else:
            x1 = x
            x += a*d
            
            b = np.dot(grad(x), grad(x) - grad(x1))/np.dot(grad(x1),grad(x1))
            #(lg.norm(grad(x - a*d))/lg.norm(grad(x)))**2
            d = -grad(x) + b*d
        
        print('  x  = ', x)
        print('f(x) = ', f(x))
        
    print('сделано ', n, ' шагов')
    
    return x

In [32]:
x0 = np.array([1.0, 1.2, 1.3])
e = 0.05
x_min = Polak_Ribier(fx, gradfx, x0, e)
print(round(fx(x_min), 3), x_min)


------[---- 1 ----]------
  x  =  [0.03108913 0.03730695 0.04041586]
f(x) =  1.003999762258978
сделано  1  шагов
1.004 [0.03108913 0.03730695 0.04041586]


In [38]:
x0 = np.array([5.,1.])
e = 0.05
x_min = Polak_Ribier(gx, gradgx, x0, e)
print(round(gx(x_min), 3), x_min)


------[---- 1 ----]------
  x  =  [0.02030533 0.88934012]
f(x) =  0.7946366053438533

------[---- 2 ----]------
  x  =  [-0.11761858  0.21813566]
f(x) =  0.17209034285214284

------[---- 3 ----]------
  x  =  [0.00388943 0.19309688]
f(x) =  0.03742255397659784
сделано  3  шагов
0.037 [0.00388943 0.19309688]


In [39]:
x0 = np.array([5.,1.,1.])
e = 0.05
x_min = Polak_Ribier(hx, gradhx, x0, e)
print(round(hx(x_min), 3), x_min)


------[---- 1 ----]------
  x  =  [-4.97003767e-07  5.99246549e-03  5.99246549e-03]
f(x) =  1.0000718218643334
сделано  1  шагов
1.0 [-4.97003767e-10  5.99246549e-03  5.99246549e-03]
